# 🌊 **Analyzing the North Atlantic Ocean by Latitude**

This notebook analyzes chlorophyll-a levels within the **North Atlantic Ocean** by latitude regions. We will use global latitude bands to divide the ocean into horizontal slices (e.g., 30°N–40°N), then calculate daily chlorophyll-a concentration trends within each slice.

The workflow includes:
- Loading the **GOaS ocean boundaries** and **latitude band** datasets.
- Clipping the **North Atlantic** region by each latitude band.
- Extracting **daily chlorophyll values** per band from a NetCDF raster.
- Exporting a time series of chlorophyll means and generating plots.

## 📑 Table of Contents
- [🧰 1. Import Required Libraries](#1-import-required-libraries)  
- [🌊 2. Load Ocean Vector (GOaS)](#2-load-ocean-vector-goas)  
- [🗺️ 3. Load Latitude Bands](#3-load-latitude-bands)  
- [✂️ 4. Clip Ocean by Latitude](#4-clip-ocean-by-latitude)  
- [📈 5. Extract Chlorophyll Data](#5-extract-chlorophyll-data)  
- [📊 6. Visualize Time Series](#6-visualize-time-series)
- [🧭 7. Next Steps](#7-next-steps)


## 🧰 **1. Import Required Libraries**

- **Geospatial libraries**  
  For working with map data like polygons and raster files:  
  - `geopandas` loads and manipulates vector data (e.g., shapefiles, GeoJSON).  
  - `shapely` helps define and clean up geometric shapes.  
  - `rasterio` and `rioxarray` let us read and clip NetCDF raster files (like chlorophyll data).  

- **Data handling and computation**  
  For loading, cleaning, and processing data efficiently:  
  - `pandas` handles tabular data.  
  - `numpy`, `xarray`, and `dask` allow large, multi-dimensional data processing in chunks.  

- **Plotting libraries**  
  To visualize chlorophyll concentrations and trends:  
  - `matplotlib.pyplot` draws plots and maps.  
  - `matplotlib.dates` formats date-based x-axes.  
  - `matplotlib.cm` provides access to color palettes.  

- **Utilities**  
  For smooth workflows and progress tracking:  
  - `tqdm` shows progress bars
  - `pathlib` handles file paths across systems.  
  - `re` helps clean up and format text (like band labels).
  - `loguru` displays clean and colorful logging messages (e.g., 🎉 success, 🚩 error).

In [ ]:
# --- Geospatial libraries ---
import geopandas as gpd  # Geographic vector data (GeoDataFrames)
from shapely.geometry import Polygon  # Creating polygon geometries
import rasterio  # Raster file I/O and metadata
import rasterio.mask  # Masking raster data with geometries
import rioxarray as rxr  # Geospatial extension for xarray (CRS-aware raster I/O)

# --- Data handling and computation ---
import pandas as pd  # Tabular data manipulation (DataFrames)
import numpy as np  # Numerical operations
import xarray as xr  # Multi-dimensional labeled arrays (NetCDF, climate, satellite)

# --- Plotting libraries ---
import matplotlib.pyplot as plt  # Core plotting API
from matplotlib.dates import MonthLocator, DateFormatter, AutoDateLocator, ConciseDateFormatter  # Date-based x-axis formatting
from matplotlib import colormaps  # Access to colormaps for styling plots

# --- Utilities ---
from pathlib import Path  # Object-oriented filesystem paths
import re  # Regular expressions for string parsing and pattern matching
from loguru import logger  # Simple, colorful logging
import sys  # For configuring logging output
from tqdm.notebook import tqdm # Progress bars

# Set up the logger to show clean messages (colored text, no background)
from helpers import configure_logger
configure_logger()

# Check that everything was imported successfully
logger.success("🎉 Libraries successfully imported.")

## 🌊 **2. Load Ocean Vector (GOaS)**

Before proceeding, please download the **Global Oceans and Seas (GOaS)** dataset from the following link:  
   [Download GOaS_v1_20211214_gpkg.zip](https://www.marineregions.org/download_file.php?name=GOaS_v1_20211214_gpkg.zip)

   Once downloaded, **extract** the ZIP file and point to the `.gpkg` file in the cell below.

In [ ]:
try:
    # Replace with the path to your downloaded GOaS dataset
    goas_vector = gpd.read_file("/Users/chiara/personal_projects/250226_Plankton/GOaS_v1_20211214_gpkg/goas_v01.gpkg")
    logger.success("🎉 GOaS dataset successfully loaded into GeoDataFrame.")
except Exception as e:
    logger.error(f"🚩 Failed to load GOaS dataset: {e}. Did you download the file manually yet?")

### 2.1: Filter Ocean Vector
In this step, we apply a **filter** to the `goas_vector` **GeoDataFrame** to select features that match the **"North Atlantic Ocean"** in the `"name"` column.

- **Setting the Filter**: We define the `filter_name` variable as `"North Atlantic Ocean"`, which will be used to filter the `goas_vector` dataset.
  
- **Filtering**: We use `str.contains()` with the filter string to **select** only those rows where the `"name"` column contains the filter value, ensuring case-insensitivity and excluding `NaN` values.

This ensures that only relevant features (those matching "North Atlantic Ocean") are retained for further processing.


In [ ]:
try:
    # Set the filter for "North Atlantic Ocean"
    filter_name = "North Atlantic Ocean"

    logger.info(f"Filtering GOaS vector for 'name' matching '{filter_name}'")
    goas_vector = goas_vector[goas_vector["name"].str.contains(filter_name, case=False, na=False)]

    if goas_vector.empty:
        logger.warning(f"⚠️ No features found matching 'name' with '{filter_name}'. Exiting.")
        raise ValueError(f"No features found for {filter_name}.")
    else:
        logger.success(f"🎉 Successfully filtered GOaS vector for 'name' matching '{filter_name}'.")

except Exception as filter_error:
    logger.error(f"🚩 Filtering error: {filter_error}")
    raise filter_error

## 🗺️ **3. Load Latitude Bands**

Next, we load the **latitude bands** that we created in "[01_create_latitude_blocks](01_create_latitude_blocks.ipynb)" to clip the ocean vector. These latitude bands will help us analyze one section of the ocean at a time, as different latitudes experience phytoplankton blooms at different times.

In [ ]:
try:
    # Replace with the path to the latitude bands GeoJSON file
    clip_vector = gpd.read_file(Path.cwd()/"data/latitude_bands_global.geojson")
    
    logger.success("🎉 Latitude bands successfully loaded into GeoDataFrame.")
except Exception as e:
    logger.error(f"🚩 Failed to load latitude bands: {e}")

## ✂️ **4. Clip Ocean by Latitude**

This step performs the clipping of the **GOaS vector** by each latitude band. For each latitude band:

- We clean the **band label** to make it a valid filename by removing special characters and replacing spaces with underscores.
- We use **GeoPandas’ `clip` function** to clip the ocean data based on the latitude band’s geometry.

In [ ]:
try:
    all_clipped_geometries = []  # List to store clipped geometries
    band_labels = []  # List to store corresponding band labels
    
    # Iterate through each latitude band and clip
    for _, row in clip_vector.iterrows():
        clip_geom = row.geometry
        band_label = row["Latitude_Range"]

        # Clean the band label: Replace spaces with "_", slashes with "-", and remove degree signs
        band_label = re.sub(r"[°]", "", band_label)  # Remove degree signs
        band_label = band_label.replace(" ", "_").replace("/", "-")

        logger.info(f"Clipping for latitude range {band_label}...")

        # Use GeoPandas `clip` function to clip the GOaS vector by the latitude band geometry
        clipped_gdf = gpd.clip(goas_vector, clip_geom)

        # Add to the list if not empty
        if not clipped_gdf.empty:
            all_clipped_geometries.append(clipped_gdf)  # Collect clipped data
            band_labels.append(band_label)  # Collect the corresponding band label
            logger.success(f"🎉 Clipped data for {band_label} added.")
        else:
            logger.warning(f"⚠️ No features found for latitude range {band_label}. This could be due to {filter_name} not occurring in this latitude range. Skipping.")

    # Combine all clipped geometries into a single GeoDataFrame
    if all_clipped_geometries:
        clipped_gdf = gpd.GeoDataFrame(pd.concat(all_clipped_geometries, ignore_index=True))
        clipped_gdf['Latitude_Range'] = band_labels  # Add latitude band labels to the combined GeoDataFrame

        # Log a success message with some stats
        logger.success(
            f"🎉 Successfully clipped {len(all_clipped_geometries)} latitude bands "
            f"with a total of {len(clipped_gdf)} features."
        )
    else:
        logger.warning("⚠️ No features were clipped. No file created.")
except Exception as e:
    logger.error(f"🚩 Error during the clipping process: {e}")

## 📈 **5. Extract Chlorophyll Data**

In [ ]:
# File paths
zarr_files = list(Path("data").glob("*.zarr"))

if not zarr_files:
    logger.error("🚩 No zarr files found.")
elif len(zarr_files) == 1:
    zarr_file = zarr_files[0]
    logger.success(f"🎉 Found one zarr file: {zarr_file}")
else:
    logger.warning("⚠️ Multiple zarr files found:")
    for i, f in enumerate(zarr_files, 1):
        logger.info(f"{i}: {f.name}")
    zarr_file = zarr_files[int(input("Pick a file number: ")) - 1]
    logger.success(f"🎉 Using: {zarr_file}")

In [ ]:
# Open zarr file
zarr = xr.open_zarr(zarr_file)
# Ensure the Zarr has a CRS
zarr = zarr.rio.write_crs("EPSG:4326")
zarr

In [ ]:
# Select "chl" data variable
chlorophyll_dataset = zarr.chl
chlorophyll_dataset

In [ ]:
# Create a list to store results for each band
band_results = []

# Process all latitude bands
logger.info(f"Processing {len(clipped_gdf)} latitude bands...")

# Create a single progress bar for the entire process
for idx, row in tqdm(clipped_gdf.iterrows(), total=len(clipped_gdf), 
                     desc="Processing latitude bands", leave=True):
    try:
        geom = row.geometry
        
        # Get the bounds of the geometry to use for the band name
        bounds = geom.bounds
        
        # Clip the dataset to the geometry
        clipped_data = chlorophyll_dataset.rio.clip([geom], drop=True)
            
        # Calculate daily mean across the spatial dimensions
        daily_mean = clipped_data.mean(dim=['longitude', 'latitude'])
        
        # Convert to dataframe
        df = daily_mean.to_dataframe().reset_index()
        
        # Add latitude band information
        lat_range = f"{int(bounds[1])}-{int(bounds[3])}°N"
        df['lat_band'] = lat_range
        
        # Append to results
        band_results.append(df)
        logger.info(f"🎉 Successfully processed {len(band_results)} out of {len(clipped_gdf)} bands.")
    except Exception as e:
        logger.error(f"🚩 Error processing band {idx}: {e}")
        continue

## 📊 **6. Visualize time-series**

In [ ]:
# Combine all dataframes from band_results
all_results = pd.concat(band_results)

# Create pivot table with time as index and lat_band as columns
pivot_data = all_results.pivot_table(index='time', columns='lat_band', values='chl')

# Sort by time to ensure smooth plotting
pivot_data = pivot_data.sort_index()

# --- Plot setup ---
fig, ax = plt.subplots(figsize=(14, 7))  # Define figure size

# Set up a colormap with as many colors as there are latitude bands
cmap = plt.get_cmap("Set2", len(pivot_data.columns))

# Plot each latitude band
for i, col in enumerate(pivot_data.columns):
    ax.plot(
        pivot_data.index,
        pivot_data[col],
        label=col,  # Format is already "X-Y°N"
        color=cmap(i),
        linewidth=2
    )

# --- Titles and labels ---
plt.suptitle(
    "Chlorophyll-a Concentration in the North Atlantic",
    fontsize=18, fontweight="bold", x=0.01, ha='left'
)
plt.title(
    "Average daily chlorophyll-a concentration by latitude",
    fontsize=12, loc='left'
)

# --- X-axis: auto-formatting for any time range ---
ax.xaxis.set_major_locator(AutoDateLocator())
ax.xaxis.set_major_formatter(ConciseDateFormatter(AutoDateLocator()))
plt.setp(ax.get_xticklabels(), rotation=45, ha="right")  # Rotate labels for readability

# Add y-axis label
ax.set_ylabel("Chlorophyll-a (mg/m³)", fontsize=12)

# --- Styling ---
ax.legend(title="", loc="upper left", fontsize=10)
ax.tick_params(axis='both', labelsize=10)
ax.grid(False)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)

plt.tight_layout()
plt.show()

## 🧭 **7. Next Steps**

After exploring the chlorophyll-a patterns by latitude, you may notice that chlorophyll-a concentrations below 30°N remained relatively stable throughout the year. To understand this a bit more, I consulted with an ocean scientist, Elodie Gutknecht at Mercator Ocean. She explained that the North Atlantic Spring Bloom generally occurs between **30° and 70°N**, and is defined by a **sharp spike in phytoplankton concentrations** over a short period of time.

With this in mind, we can now focus our map visualization on the **2024 Spring Bloom Season** of (March to June) and a **smaller geographic window** that captures the most relevant activity.

In the next notebook, we'll extract daily chlorophyll-a timesteps from our Zarr file and **export each one as a GeoTIFF**, allowing us to style and animate bloom dynamics in QGIS.

Continue to:  
👉 [`03_zarr_to_geotiffs.ipynb`](03_zarr_to_geotiffs.ipynb)
